<a href="https://colab.research.google.com/github/prabinrs/ku_hmis_205/blob/patch-1/notebook/lab02_ecg_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: ECG Signal Loading and Visualization
## Medical Signal Processing & Telemedicine Integration
---

## Learning Objectives
By the end of this lab, you will be able to:
1. Load ECG waveforms from MIMIC-III Waveform Database
2. Understand ECG signal characteristics (sampling rate, leads, duration)
3. Create professional medical signal visualizations
4. Identify basic ECG components (P, QRS, T waves)

---

## Setup

In [ ]:
# @title Student Information
Stuent_Name = "" # @param {"type":"string","placeholder":"Enter your Name"}
Student_ID = "" # @param {"type":"string","placeholder":"Eneter your ID"}
Date = "2025-11-19" # @param {"type":"date","placeholder":"Today's Date"}

# Setup

In [ ]:
! pip install WFDB

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from IPython.display import display

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (15, 8)

print("✓ Libraries imported successfully")
print(f"WFDB version: {wfdb.__version__}")

---

## Part 1: Loading ECG Data

We'll use the MIMIC-III Waveform Database Matched Subset. This contains continuous waveform recordings from ICU patients.
link : https://physionet.org/content/mimic3wdb-matched/1.0/

The total size is around 2 TB, so for the class we will download 10 to 15 patient's data only.

**Optional**
If you want to use the dataset to create Machine Learning Project, there are multiple annotation are avaliable:

for AF : https://figshare.com/articles/dataset/Atrial_Fibrillation_annotations_of_electrocardiogram_from_MIMIC_III_matched_subset/12149091?file=22340070 (for the old version)

for Sepesis : https://figshare.com/articles/dataset/ECG_sepsis_xlsx/24265717?file=42594220


### Background: MIMIC-III Waveform Structure
- **Directory structure:** `physionet.org/files/mimic3wdb-matched/1.o/p00/p0000XX/`
- **Files per record:**
  - `.hea` - Header file with metadata
  - `.dat` - Binary signal data
  - `.atr` - Annotations (if available)




### Download files to local drive (colab drive):

I prefer bash command directly from the notebook to download the data to root folder while using colab, but you are free to download manually if you are using local machine.


Play with the list of subfolders , here are some list of subfolers in p00 folder. Feel free to check pysionet's site and make changes, you can also use other folders p01 p02 .... p09



SUBFOLDERS=(
  "p000020"
  "p000030"
  "p000033"
  "p000052"
  "p000079"
  "p000085"
  "p000107"
  "p000109"
  "p000123"
  "p000124"
  "p000125"
  "p000135"
  "p000138"
  "p000145"
  "p000154"
  "p000160"
  "p000177")

  ### Good and Bad Examples of ECG:
  Good : /p00/p000030/3524877_0001

  Bad : /poo/p000020/3544749_0001

In [ ]:
%%bash

# Define the exact list of subfolders you want
SUBFOLDERS=( ) # list of subfolders you wanna download


# This is the URL for the p00 folder
BASE_URL="https://physionet.org/files/mimic3wdb-matched/1.0/p00/"

# Loop through each subfolder and download it recursively
for FOLDER in "${SUBFOLDERS[@]}"
do
  echo "--- Downloading ${FOLDER} ---"
  # We add the folder name to the base URL to get the full path
  wget -r -N -c -np "${BASE_URL}${FOLDER}/"
done

echo "--- All specific folders downloaded! ---"

### Exercise 1.1: Load a Single ECG Record

In [ ]:
# Load example record from MIMIC-III Waveform Database

record_name = '3544749_0001'
pn_dir = '/content/physionet.org/files/mimic3wdb-matched/1.0/p00/p000020/'

# TODO: Load the record using wfdb.rdrecord()
# Hint: record = wfdb.rdrecord(fullPath)
record = # YOUR CODE HERE

# Display basic information
print("=" * 60)
print("ECG RECORD INFORMATION")
print("=" * 60)
print(f"Record Name: {record.record_name}")
print(f"Sampling Frequency: {record.fs} Hz")
print(f"Number of Signals: {record.n_sig}")
print(f"Signal Names: {record.sig_name}")
print(f"Duration: {record.sig_len / record.fs:.2f} seconds")
print(f"Signal Shape: {record.p_signal.shape}")

**Question 1.1:** What do the dimensions of `record.p_signal.shape` represent?

**Your Answer:**  
_Write your answer here_

---

### Exercise 1.2: Explore Signal Metadata

In [ ]:
# Extract and display detailed signal information
print("\nDETAILED SIGNAL INFORMATION:")
print("-" * 60)

for i in range(record.n_sig):
    print(f"\nSignal {i}: {record.sig_name[i]}")
    print(f"  Units: {record.units[i]}")
    print(f"  Gain: {record.adc_gain[i]}")
    print(f"  Baseline: {record.baseline[i]}")

    # TODO: Calculate and display basic statistics for each signal
    # Hint: Use numpy functions like mean, std, min, max
    signal_data = record.p_signal[:, i]
    print(f"  Mean: {np.mean(signal_data):.3f}")
    print(f"  Std Dev: # YOUR CODE HERE")
    print(f"  Range: [{np.min(signal_data):.3f}, {np.max(signal_data):.3f}]")

---

## Part 2: Basic ECG Visualization

### Exercise 2.1: Plot a Single Lead

In [ ]:
# Let's visualize Lead II (typically at index 1)
lead_index = 1
lead_name = record.sig_name[lead_index]

# Extract 10 seconds of data
duration_seconds = 10
start_sample = 0
end_sample = int(duration_seconds * record.fs)

# TODO: Extract the signal segment and time array
ecg_segment = record.p_signal[start_sample:end_sample, lead_index]
time_array = # YOUR CODE HERE (hint: use np.arange and divide by sampling frequency)

# Create the plot
plt.figure(figsize=(15, 5))
plt.plot(time_array, ecg_segment, 'b-', linewidth=0.5)
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel(f'Amplitude ({record.units[lead_index]})', fontsize=12)
plt.title(f'ECG Signal - {lead_name} (First {duration_seconds} seconds)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Question 2.1:** Approximately how many heartbeats do you see in this 10-second window?

**Your Answer:**  
_Write your answer here_

---

### Exercise 2.2: Professional Multi-Lead Visualization

In [ ]:
def plot_multi_lead_ecg(record, start_time=0, duration=10, leads=None):
    """
    Create a professional multi-lead ECG plot

    Parameters:
    -----------
    record : wfdb.Record
        WFDB record object
    start_time : float
        Starting time in seconds
    duration : float
        Duration to plot in seconds
    leads : list or None
        List of lead indices to plot (None = all leads)
    """
    fs = record.fs
    start_sample = int(start_time * fs)
    end_sample = int((start_time + duration) * fs)

    if leads is None:
        leads = range(record.n_sig)

    # TODO: Create time array
    time_array = # YOUR CODE HERE

    # Create subplots
    fig, axes = plt.subplots(len(leads), 1, figsize=(15, 3*len(leads)))
    if len(leads) == 1:
        axes = [axes]

    # TODO: Plot each lead
    for idx, lead in enumerate(leads):
        signal = record.p_signal[start_sample:end_sample, lead]

        axes[idx].plot(time_array, signal, 'k-', linewidth=0.8)
        axes[idx].set_ylabel(f"{record.sig_name[lead]}\n({record.units[lead]})", fontsize=10)
        axes[idx].grid(True, alpha=0.3)
        axes[idx].set_xlim(time_array[0], time_array[-1])

    axes[-1].set_xlabel('Time (seconds)', fontsize=12)
    fig.suptitle(f'Multi-Lead ECG - Record {record.record_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Test the function
plot_multi_lead_ecg(record, start_time=0, duration=10, leads=[0, 1, 2])

---

## Part 3: Interactive Exploration

### Exercise 3.1: Explore Different Time Windows

In [ ]:
# TODO: Modify these parameters to explore different parts of the recording
start_time = 0  # seconds
duration = 5    # seconds
lead_to_plot = 1  # Lead II

# YOUR CODE HERE: Create a plot of the specified segment
# Use the code from Exercise 2.1 as a template

### Exercise 3.2: Identify ECG Components

Find a clear heartbeat in your plot and try to identify:
- **P wave:** Small bump before the QRS complex (atrial depolarization)
- **QRS complex:** Large spike (ventricular depolarization)
- **T wave:** Rounded wave after QRS (ventricular repolarization)

**Instructions:**
1. Plot a 2-3 second window with 1-2 clear heartbeats
2. Add vertical lines or annotations to mark P, QRS, and T waves
3. Calculate the approximate duration of the QRS complex

In [ ]:
# TODO: Complete this exercise
# Example: Plot and annotate one complete cardiac cycle

# Select a short segment (2-3 seconds)
start = 0
duration = 3
fs = record.fs

# Extract segment
start_idx = int(start * fs)
end_idx = int((start + duration) * fs)
ecg = record.p_signal[start_idx:end_idx, 1]
time = np.arange(len(ecg)) / fs

# Plot
plt.figure(figsize=(15, 6))
plt.plot(time, ecg, 'b-', linewidth=1)

# TODO: Add annotations for P, QRS, T waves
# Example: plt.axvline(x=0.2, color='r', linestyle='--', label='P wave')
# YOUR CODE HERE

plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude (mV)')
plt.title('ECG Components Identification')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

**Question 3.2:**
1. What is the approximate duration of the QRS complex you identified? (Normal: 80-120 ms)
2. Is this within normal limits?

**Your Answer:**  
_Write your answer here_

---

## Part 4: Signal Statistics and Analysis

### Exercise 4.1: Calculate Heart Rate

In [ ]:
# Manual heart rate estimation
# Count number of QRS complexes in a 10-second window

duration_seconds = 10
num_beats = 0  # TODO: Count the beats manually from your plot

# TODO: Calculate heart rate
heart_rate_bpm = (num_beats / duration_seconds) * 60

print(f"Estimated Heart Rate: {heart_rate_bpm:.1f} bpm")
print(f"Normal Range: 60-100 bpm")
print(f"Status: {'Normal' if 60 <= heart_rate_bpm <= 100 else 'Abnormal'}")

### Exercise 4.2: Signal Quality Assessment

In [ ]:
# Calculate basic quality metrics
ecg_full = record.p_signal[:, 1]  # Lead II

# TODO: Calculate signal statistics
signal_mean = np.mean(ecg_full)
signal_std = # YOUR CODE HERE
signal_range = np.max(ecg_full) - np.min(ecg_full)

# Check for flat lines (poor quality indicator)
diff_signal = np.abs(np.diff(ecg_full))
flat_ratio = np.sum(diff_signal < 0.001) / len(diff_signal)

print("SIGNAL QUALITY ASSESSMENT")
print("=" * 50)
print(f"Mean: {signal_mean:.3f} mV")
print(f"Standard Deviation: {signal_std:.3f} mV")
print(f"Signal Range: {signal_range:.3f} mV")
print(f"Flat Line Ratio: {flat_ratio*100:.2f}%")
print(f"\nQuality: {'Good' if flat_ratio < 0.05 else 'Poor'}")

---

## Part 5: Comparison Across Leads

### Exercise 5.1: Compare Different Leads

In [ ]:
# TODO: Plot the same time segment for all available leads
# Compare amplitude and morphology

plot_multi_lead_ecg(record, start_time=0, duration=5)

**Question 5.1:** Which lead shows the highest amplitude QRS complexes? Why might this be?

**Your Answer:**  
_Write your answer here_

---

---

## Reflection

**Question:** What did you learn from this lab? What was most challenging?

**Your Response:**  
_Write your reflection here (3-5 sentences)_

---

## References

1. Goldberger, A., et al. (2000). PhysioBank, PhysioToolkit, and PhysioNet. Circulation. 101(23):e215-e220.
2. Johnson, A.E.W., et al. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.
3. WFDB Python Package: https://wfdb.readthedocs.io/

---

**End of Lab 2**